# CatBoost

In [14]:
from catboost import CatBoostRegressor
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool

In [15]:
# Inserting features from TRAIN into one dataset

folder_train = '../features_train_in_use'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)
print(features_train)

       average_income_per_grunnkrets  numberOfBusstop categorical_chain_name  \
0                           561700.0             10.0              MCDONALDS   
1                           555720.0             30.0              MCDONALDS   
2                           495900.0             14.0            BURGER KING   
3                           521240.0             11.0            BURGER KING   
4                           585360.0             29.0            BURGER KING   
...                              ...              ...                    ...   
12854                       505640.0             27.0                    NaN   
12855                       396640.0             12.0       GULATING GRUPPEN   
12856                       549120.0             13.0       GULATING GRUPPEN   
12857                       498660.0             35.0                    NaN   
12858                       579780.0             10.0                    NaN   

      categorical_mall_name categorical

In [16]:
# Inserting features from TEST into one dataset

folder_test = '../features_test_in_use'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)
print(features_test)

      average_income_per_grunnkrets  numberOfBusstop categorical_chain_name  \
0                          551600.0             21.0                    NaN   
1                          531240.0             11.0            BURGER KING   
2                          523300.0             11.0            BURGER KING   
3                          591380.0             22.0            BURGER KING   
4                          594020.0             21.0       VULKAN BURGERBAR   
...                             ...              ...                    ...   
8572                       560700.0              9.0       GULATING GRUPPEN   
8573                       593340.0             28.0                    NaN   
8574                       629100.0              6.0       GULATING GRUPPEN   
8575                       590100.0              8.0                    NaN   
8576                       529980.0              4.0                    NaN   

     categorical_mall_name categorical_plaace categ

In [17]:
# Target TRAIN data

target_train = pd.read_csv("../data/stores_train.csv")
target_train = target_train['revenue'].values

target_train = np.log1p(target_train)

In [18]:
catboost_model = CatBoostRegressor(
    n_estimators=2000,
    learning_rate=0.05,
    thread_count=-1,
    max_depth=6,
    silent=True,
    loss_function='RMSE',
    random_seed=42,
    bagging_temperature=0.3,
    od_type="Iter"
)

In [19]:
for column in features_train.columns:
    values=features_train[column].values
    for i in range(len(values)):
        if type(values[i])==float:
            values[i]="nan"

In [20]:
for column in features_test.columns:
    values=features_test[column].values
    for i in range(len(values)):
        if type(values[i])==float:
            values[i]="nan"

In [21]:
# TRAIN

categorical_features_train = []

for col in features_train.columns:
    if col.startswith("categorical"):
        categorical_features_train.append(col)

Pool_train= Pool(features_train,target_train,cat_features=categorical_features_train)

c:\Python310\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [22]:
# TEST

categorical_features_test = []

for col in features_test.columns:
    if col.startswith("categorical"):
        categorical_features_test.append(col)

Pool_test= Pool(features_test, cat_features=categorical_features_test)

In [23]:
# Fit and predict

catboost_model.fit(Pool_train)

catboost_predictions = catboost_model.predict(Pool_test)
catboost_predictions = np.expm1(catboost_predictions)




In [24]:
print(catboost_predictions)

[4.42105707 9.79808928 2.74481924 ... 3.7469135  3.7331962  1.97785286]


In [25]:
# Get storeid as first column

stores_test = pd.read_csv("../data/stores_test.csv")
stores_test_id = stores_test['store_id'].values

# insert predictions as second column

predictions = pd.DataFrame()

predictions['id'] = stores_test_id
predictions['predicted'] = catboost_predictions
predictions.to_csv("../predictions/catboost5.csv", index=False)
print(predictions)

                              id  predicted
0     914206820-914239427-717245   4.421057
1     916789157-916823770-824309   9.798089
2       913341082-977479363-2948   2.744819
3      889682582-889697172-28720  11.072728
4     997991699-998006945-417222  14.007197
...                          ...        ...
8572  917323003-917383529-844309   3.119592
8573  917353379-917411824-845904   3.258263
8574  917072302-917089248-833647   3.746914
8575  916960557-916993161-829908   3.733196
8576   987280891-972040746-45320   1.977853

[8577 rows x 2 columns]


In [26]:
def rmsle(y_true, y_pred):
    """
    Computes the Root Mean Squared Logarithmic Error 
    
    Args:
        y_true (np.array): n-dimensional vector of ground-truth values 
        y_pred (np.array): n-dimensional vecotr of predicted values 
    
    Returns:
        A scalar float with the rmsle value 
    
    Note: You can alternatively use sklearn and just do: 
        sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    """
    assert (y_true >= 0).all(), 'Received negative y_true values'
    assert (y_pred >= 0).all(), 'Received negative y_pred values'
    assert y_true.shape == y_pred.shape, 'y_true and y_pred have different shapes'
    y_true_log1p = np.log1p(y_true)  # log(1 + y_true)
    y_pred_log1p = np.log1p(y_pred)  # log(1 + y_pred)
    return np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p)))


X_train, X_test, y_train, y_test = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

catboost_model2 = CatBoostRegressor(
    n_estimators=2000,
    learning_rate=0.05,
    thread_count=-1,
    max_depth=6,
    silent=True,
    loss_function='RMSE',
    random_seed=42,
    bagging_temperature=0.3,
    od_type="Iter"
)

catboost_model2.fit(X_train, y_train)
y_pred = catboost_model2.predict(X_test)


# Calculate rmsle for a few example predictions 
y_true = y_test
n = len(features_train)
print('A couple of RMSLE scores computed over the train set')
print(f'Perfect prediction: {rmsle(y_true, y_true):.4f}')
print(f'All zeros prediciton: {rmsle(y_true, y_pred):.4f}')
print(f'All ones prediction: {rmsle(y_true, y_pred):.4f}')

c:\Python310\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


CatBoostError: Bad value for num_feature[non_default_doc_idx=2,feature_idx=2]="B-YOUNG": Cannot convert 'b'B-YOUNG'' to float